# Cleaning Scholars@TAMU Data  
**Filename:** cleaning.ipynb  
**Path:** TAMIDS/Code/Scholars@TAMU Data/cleaning.ipynb  
**Created Date:** 02 April 2022, 20:07 

The data provided in .xlsx format has some issues. It seems like wherever a comma occurs in these files, the columns get split up. Likely because these were originally .csv files delimmited by commas. Another issue is that some columns are interpreted by pandas as having multiple data types. This notebook should rectify these issues.

In [123]:
from IPython.display import Markdown, display, HTML
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

# pd.options.display.float_format = '{:,.3f}'.format
pd.options.display.max_columns = 100
plt.style.use('seaborn-darkgrid')

# General Markdown Formatting Functions

def printmd(string, level=1):
    header_level = '#'*level + ' '
    display(Markdown(header_level + string))

## Scholars@TAMU

In [124]:
base_path = "../../Data/Scholars@TAMU"

!tree "$base_path"

zsh:1: command not found: tree


In [125]:
with open('dicts/raw_filenames.json', 'r') as outfile:
    raw_filenames = json.load(outfile)

# make a .json file of the cleaned data filenames that is identical to the raw filenames
data_filenames = raw_filenames

with open('dicts/data_filenames.json', 'w') as outfile:
    json.dump(data_filenames, outfile, indent=4)

### Generally Helpful Functions

In [126]:
def fix_trailing_columns(df: pd.DataFrame, last_column: str) -> pd.DataFrame:
    """The is a common problem of the last column of the DataFrames being split into multiple columns with the name 'Unnamed: ##'. 
    Every split represents where a comma should be, this function fixes that."""
    trailing_columns=df.copy()
    abstract = trailing_columns[last_column].copy()

    for column in trailing_columns.loc[:, last_column:]:
        if column == last_column:
            continue
        
        str_column = trailing_columns[column].apply(str)
        str_column = ',' + str_column
        str_column[str_column == ',nan'] = ''
        abstract += str_column

    df = df.loc[:, :last_column]
    df[last_column] = abstract
    return df

def rows_of_type(df: pd.DataFrame, column_name: str, of_type: type) -> pd.DataFrame:
    """This function returns a dataframe that has cells in {column name} that are of type {of_type}."""
    return df[df[column_name].transform(type) == of_type]

## ./people

In [127]:
people_path = base_path + "/people"

people_data = {picklename: pd.read_pickle(people_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['people']}

### people_education

In `df.['year']` each cell is a year of type `int`. Some cells have extra characters making them a string.

In [128]:
# I wrote the following function like this expecting for this to be a recurring problem
def fix_int_with_letters(int_string: str, substring: str) -> int | str:
    if substring in int_string:
        int_string = int_string.replace(substring, '')
        return int(int_string)
    else:
        return int_string

def clean_education_year(cell: object) -> int | object:
    if type(cell) == str:
        cell = fix_int_with_letters(int_string=cell, substring='_x000D_')
    return cell
 

people_data['people_education']['year'] = people_data['people_education']['year'].apply(clean_education_year)

# pandas likes to upcast `int` to `float`
people_data['people_education']['year'] = people_data['people_education']['year'].astype('Int64')


### people_overview

In `df.['overview']` new columns get added for every comma in the string. These must be concatenated.

In [129]:
people_data['people_overview'] = fix_trailing_columns(df=people_data['people_overview'], last_column='overview')

rows_of_type(df=people_data['people_overview'], column_name='uin', of_type=str)
people_data['people_overview'] = people_data['people_overview'].drop(830)

### .to_pickle

In [130]:
for (key, df), picklefile in zip(people_data.items(), data_filenames['people']):
    filepath = f"{people_path}/{picklefile}.pickle"
    df.to_pickle(filepath)

    print(f"people_data[{key}]\t->\t{people_path}/{picklefile}.pickle")

people_data[people_affiliation]	->	../../Data/Scholars@TAMU/people/people_affiliation.pickle
people_data[people_education]	->	../../Data/Scholars@TAMU/people/people_education.pickle
people_data[people_overview]	->	../../Data/Scholars@TAMU/people/people_overview.pickle
people_data[people_subject_areas]	->	../../Data/Scholars@TAMU/people/people_subject_areas.pickle


## ./courses

In [131]:
courses_path = base_path + "/courses"

courses_data = {picklename: pd.read_pickle(courses_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['courses']}

### course_overview

In [132]:
printmd('Rows with an invalid `course_number`', 0)
display(rows_of_type(df=courses_data['course_overview'], column_name='course_name', of_type=int))
# row [18409] was dropped because 'Immunology 6740' is not a valid class code for TAMU
courses_data['course_overview'] = courses_data['course_overview'].drop(18409)

printmd('Rows with a range of years (`[int]`) in the `year` column instead of each individual year (`int`)', 0)
display(rows_of_type(df=courses_data['course_overview'], column_name='year', of_type=str))

year_range_df = rows_of_type(df=courses_data['course_overview'], column_name='year', of_type=str)

# this dictionary wasn't automated because it wasn't needed for this size
year_ranges = {
    '15947': (2016, 2017),
    '19494': (2008, 2019),
    '19495': (2016, 2019),
    '20672': (2016, 2017),
    '21117': (2015, 2019)
}

for k, v in year_ranges.items():
    base_row = courses_data['course_overview'].iloc[int(k)].copy()
    for year in range(v[0], v[1] + 1):
        base_row['year'] = year
        courses_data['course_overview'].loc[courses_data['course_overview'].shape[0]] = base_row

for k, v in year_ranges.items():
    courses_data['course_overview'].drop(int(k), inplace=True)

 Rows with an invalid `course_number`

,uid,uin,people_uri,people_api,course,course_name,acad_dept,year
18409,6384021d9c07df50570362e5f602209f,602004217,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,Immunology,6740,DDDS,2018


 Rows with a range of years (`[int]`) in the `year` column instead of each individual year (`int`)

,uid,uin,people_uri,people_api,course,course_name,acad_dept,year
15947,a42c45f158cb4087da2810141b94e077,126005712,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,MSCI601,Contem Top In Adv Cell Bio I,CLMD,2016 2017
19494,594e832eaacbca290af050d732e893fa,618009393,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,PHAR726,Microbiology/immunology,CLCP,2008-2019
19495,594e832eaacbca290af050d732e893fa,618009393,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,PHAR811,"Crtcl Care, Gi, Pulm, Rheum,",CLCP,2016-2019
20672,69e129774b67b31ed1b01ad8340d8734,821002114,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,PHAR842,Patient Assessment,CLCP,2016 2017
21117,ebcae5bf949310d335e3324c64d808c5,420002948,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,GENE608,Crit Analysis Gene Lit,BCBP,2015-2019


In [133]:
for (key, df), picklefile in zip(courses_data.items(), data_filenames['courses']):
    filepath = f"{courses_path}/{picklefile}.pickle"
    df.to_pickle(filepath)

    print(f"courses_data[{key}]\t->\t{courses_path}/{picklefile}.pickle")

courses_data[course_overview]	->	../../Data/Scholars@TAMU/courses/course_overview.pickle


## ./grants_and_awards

In [134]:
grants_path = base_path + "/grants_and_awards"

grants_data = {picklename: pd.read_pickle(grants_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['grants_and_awards']}

### awards_overview

In [135]:
printmd('The following three rows have weird data in these columns, so they will be dropped.', 0)
display(rows_of_type(df=grants_data['awards_overview'], column_name='uid', of_type=int))
display(rows_of_type(df=grants_data['awards_overview'], column_name='uin', of_type=str))
grants_data['awards_overview'] = grants_data['awards_overview'].drop(8373)
display(rows_of_type(df=grants_data['awards_overview'], column_name='award_name', of_type=int))
grants_data['awards_overview'] = grants_data['awards_overview'].drop(3145)

 The following three rows have weird data in these columns, so they will be dropped.

,uid,uin,people_uri,people_api,award_id,year,award_name,authoritative_name_id
8373,0,texas_target_communities,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,NaN,NaN,NaN,NaN


,uid,uin,people_uri,people_api,award_id,year,award_name,authoritative_name_id
8373,0,texas_target_communities,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,NaN,NaN,NaN,NaN


,uid,uin,people_uri,people_api,award_id,year,award_name,authoritative_name_id
3145,2254cbb076da3cffe94f12c8dd854f0a,202005361,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,8865.0,2015.0,2014,31420.0


### grants_abstract

In [136]:
# display(grants_data['grants_abstract'])
grants_data['grants_abstract'] = fix_trailing_columns(df=grants_data['grants_abstract'], last_column='abstract')

/var/folders/r3/xk4xn8d16wjf3qf3237rwz040000gn/T/ipykernel_4167/1832468470.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[last_column] = abstract


### grants_overview

In [137]:
grants_data['grants_overview']['grant_uid'] = grants_data['grants_overview']['grant_uid'].transform(str)

grants_data['grants_overview'] = fix_trailing_columns(df=grants_data['grants_overview'], last_column='title')

In [138]:
for (key, df), picklefile in zip(grants_data.items(), data_filenames['grants_and_awards']):
    filepath = f"{grants_path}/{picklefile}.pickle"
    df.to_pickle(filepath)

    print(f"grants_data[{key}]\t->\t{grants_path}/{picklefile}.pickle")

grants_data[awards_overview]	->	../../Data/Scholars@TAMU/grants_and_awards/awards_overview.pickle
grants_data[grants_abstract]	->	../../Data/Scholars@TAMU/grants_and_awards/grants_abstract.pickle
grants_data[grants_overview]	->	../../Data/Scholars@TAMU/grants_and_awards/grants_overview.pickle


## ./organizations

In [139]:
orgs_path = base_path + "/organizations"

orgs_data = {picklename: pd.read_pickle(orgs_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['organizations']}

### organizations_external

In [140]:
display(rows_of_type(df=orgs_data['organizations_external'], column_name='aa_id', of_type=str))
orgs_data['organizations_external'] = orgs_data['organizations_external'].drop(25524)

orgs_data['organizations_external'] = orgs_data['organizations_external'].drop(columns=['Unnamed: 9'])

,external_org_id,external_org_uri,external_org_api,authoritative_name,authoritative_name_city,authoritative_name_state,authoritative_name_country,grid_id,aa_id,Unnamed: 9
25524,25564,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,Regional Public Health Care Institution City H...,"Barnaul""""",Barnaul,NaN,Russia,grid.489152.3,NaN


In [141]:
for (key, df), picklefile in zip(orgs_data.items(), data_filenames['organizations']):
    filepath = f"{orgs_path}/{picklefile}.pickle"
    df.to_pickle(filepath)

    print(f"orgs_data[{key}]\t->\t{orgs_path}/{picklefile}.pickle")

orgs_data[organizations_external]	->	../../Data/Scholars@TAMU/organizations/organizations_external.pickle
orgs_data[organizations_local]	->	../../Data/Scholars@TAMU/organizations/organizations_local.pickle


## ./publications

In [142]:
pubs_path = base_path + "/publications"

pubs_data = {picklename: pd.read_pickle(pubs_path + "/raw_" + picklename + ".pickle") for picklename in raw_filenames['publications']}

### publications_abstract

In [143]:
pubs_data['publications_abstract'] = fix_trailing_columns(df=pubs_data['publications_abstract'], last_column='abstract')
pubs_data['publications_abstract']

/var/folders/r3/xk4xn8d16wjf3qf3237rwz040000gn/T/ipykernel_4167/1832468470.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[last_column] = abstract


,publication_uri,publication_api,abstract
0,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,NaN
1,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,BACKGROUND: Heart failure is a leading cause o...
2,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,The renin-angiotensin system is a major determ...
3,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,Â© CERN 2016. This paper describes the algorit...
4,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,The peripheral benzodiazepine receptor (PBR) h...
...,...,...,...
223173,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,Major histocompatibility complex (MHC) class I...
223174,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,Cancer cells develop under immune surveillance...
223175,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,NaN
223176,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,OBJECTIVES: Interleukin-1 receptor-associated ...


### publications_author_institutions

In `df.['city']` there was a missing city name, with an `int` there instead.

In [144]:
display(rows_of_type(df=pubs_data['publications_author_institutions'], column_name='city', of_type=int))
pubs_data['publications_author_institutions']['city'].iloc[481504] = 'Kongens Lyngby'

,publication_uri,publication_api,organisation,city,country
481504,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,Danmarks Tekniske Universitet,33,Denmark


### publications_overview

In [145]:
# display(rows_of_type(df=pubs_data['publications_overview'], column_name='journal_title', of_type=int))
pubs_data['publications_overview']['journal_title'].iloc[168590] = np.nan

display(rows_of_type(df=pubs_data['publications_overview'], column_name='month', of_type=str))

shift_once_or_twice = [13416, 45994, 91108, 158313]

for i in shift_once_or_twice:
    row = pubs_data['publications_overview'].iloc[i].copy()
    row = row.to_numpy()
    if i == 158313:
        c = 1
    else:
        c=0
    row[11:13+c] = [','.join(row[11:13+c])]
    row = np.delete(row, 12)
    if i == 158313:
        row = np.delete(row, 12)
        row = np.append(row, np.nan)

    pubs_data['publications_overview'].iloc[i] = np.append(row, np.nan)
    # display(pubs_data['publications_overview'].iloc[i:i+1])



#####
pubs_data['publications_overview'][['begin_page', 'end_page', 'volume', 'issue']] = pubs_data['publications_overview'][['begin_page', 'end_page', 'volume', 'issue']].apply(str)

# display(rows_of_type(df=pubs_data['publications_overview'], column_name='publication_type', of_type=float))
invalid_titles = rows_of_type(df=pubs_data['publications_overview'], column_name='publication_title', of_type=int)
pubs_data['publications_overview'] = pubs_data['publications_overview'].drop(invalid_titles.index)

rows_of_type(df=pubs_data['publications_overview'], column_name='publication_title', of_type=int)
pubs_data['publications_overview'] = fix_trailing_columns(df=pubs_data['publications_overview'], last_column='publication_title')

/var/folders/r3/xk4xn8d16wjf3qf3237rwz040000gn/T/ipykernel_4167/239603840.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubs_data['publications_overview']['journal_title'].iloc[168590] = np.nan


,people_uid,uin,people_uri,people_api,dept_id,publication_uid,publication_uri,publication_api,doi,issn,eissn,journal_title,month,year,begin_page,end_page,volume,issue,publisher,publication_type,publication_title,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
13416,f0a2b3a8157beca6b160571989b7711e,901006572,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,130,293378SE,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,10.1201/9781420059922,NaN,NaN,Ch. 8 in L. H. Wang (Editor) Photoacoustic ima...,"CRC Press 2009, pp. 89-101.",1,2017,89,101,NaN,NaN,NaN,Journal Article,On reconstruction formulas and algorithms for ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45994,cb5632f793354548289b5d5fca835a2e,801007443,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,87,225437SE,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,10.1109/icpe.2011.5944373,2150-6078,NaN,8th International Conference on Power Electron...,"ICPE 2011-ECCE Asia""",5,2011,19,30,NaN,NaN,IEEE,Conference,Role of Power Electronics in Renewable Energy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91108,9a66941ac14f5b1d25d6ab9d66022fa6,226004448,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,523,543627SE,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,10.1109/wowmom.2019.8792986,NaN,NaN,2019 IEEE 20th International Symposium on A Wo...,"Mobile and Multimedia Networks"" (WoWMoM)""",6,2019,1,5,0,NaN,IEEE,Conference,IoT and the Risk of Internet Exposure: Risk As...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158313,4f21a8e72e56c869094a433251106c48,702001927,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,87,401618SE,https://scholars.library.tamu.edu/vivo/display...,https://api.library.tamu.edu/scholars-discover...,10.1109/apt.1993.673899,NaN,NaN,Joint International Power Conference Athens Po...,Operation and Control in Today''s Electric Po...,"APT 1993 - Proceedings""",1,1993,771,775,2,NaN,IEEE,Conference,A New Approach For Reliability Evaluation Of I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
for (key, df), picklefile in zip(pubs_data.items(), data_filenames['publications']):
    filepath = f"{pubs_path}/{picklefile}.pickle"
    df.to_pickle(filepath)

    print(f"pubs_data[{key}]\t->\t{pubs_path}/{picklefile}.pickle")

pubs_data[publications_abstract]	->	../../Data/Scholars@TAMU/publications/publications_abstract.pickle
pubs_data[publications_author_institutions]	->	../../Data/Scholars@TAMU/publications/publications_author_institutions.pickle
pubs_data[publications_overview]	->	../../Data/Scholars@TAMU/publications/publications_overview.pickle
pubs_data[publications_subject_journal_wos]	->	../../Data/Scholars@TAMU/publications/publications_subject_journal_wos.pickle
pubs_data[publications_unsdg]	->	../../Data/Scholars@TAMU/publications/publications_unsdg.pickle
